In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [3]:
import tensorflow.keras
import tensorflow as tf

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv1D, MaxPooling1D, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Activation

In [39]:
covid_df = pd.read_excel("corona value - 08-04-99.xlsx")
covid_df.columns = covid_df.iloc[1]
covid_df = covid_df.drop(covid_df.index[[0,1,2,3,4]]).reset_index().drop('index', axis=1)

In [40]:
covid_df = covid_df.dropna()
display(covid_df)
display(covid_df.describe())
display(covid_df.info())

1,Age (yr),Gender,pulse rate,respiratory rate,Tempreture,Height,Weight,BMI,Systolic blood presure,Diastolic blood pressure,...,non-corona chest CT signs,CT is requesyed,no need to CT study,HCQ,CAL,symptomatic therapy,AZT,OST,admission,No previous treatment
0,73,male,64,22,36.6,155,60,24.97,136,41,...,0,0,0,0,0,0,0,0,0,1
1,29,female,80,22,37.1,165,67,24.61,94,44,...,0,0,0,0,0,0,0,0,0,1
4,34,female,102,22,37.2,170,53,18.34,114,47,...,0,0,0,0,0,0,0,0,0,1
5,19,male,124,20,36.9,170,65,22.49,123,50,...,0,0,1,0,0,0,0,0,0,1
6,13,female,89,18,36.3,155,54,22.48,115,51,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3474,52,female,120,19,36,173,85,28.4,156,110,...,0,1,0,0,0,0,0,0,0,1
3475,75,female,92,18,37,160,50,19.53,183,117,...,0,0,0,0,0,0,0,0,0,1
3477,39,female,125,23,37,162,106,40.39,185,125,...,0,0,0,0,0,0,0,0,0,0
3478,56,male,150,24,35,168,65,23.03,231,148,...,0,0,1,0,0,1,0,0,0,0


1,Age (yr),Gender,pulse rate,respiratory rate,Tempreture,Height,Weight,BMI,Systolic blood presure,Diastolic blood pressure,...,non-corona chest CT signs,CT is requesyed,no need to CT study,HCQ,CAL,symptomatic therapy,AZT,OST,admission,No previous treatment
count,2577,2577,2577,2577,2577.0,2577,2577,2577.00,2577,2577,...,2577,2577,2577,2577,2577,2577,2577,2577,2577,2577
unique,83,2,99,22,48.0,55,91,825.00,117,90,...,1,2,2,2,2,2,2,2,2,2
top,34,male,100,20,36.5,170,80,27.68,120,80,...,0,0,0,0,0,0,0,0,0,0
freq,98,1418,128,1079,279.0,318,209,54.00,96,114,...,2577,2198,2057,2216,2568,2221,2404,2545,2514,1573


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2577 entries, 0 to 3479
Columns: 102 entries, Age (yr) to No previous treatment
dtypes: object(102)
memory usage: 2.0+ MB


None

In [41]:
cols=[i for i in covid_df.columns if i not in ["Gender"]]
covid_df['Triage result'] = covid_df['Triage result'].astype(str)
for col in cols:
    covid_df[col]=pd.to_numeric(covid_df[col])
    covid_df[col]=covid_df[col].astype(np.float32)
display(covid_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2577 entries, 0 to 3479
Columns: 102 entries, Age (yr) to No previous treatment
dtypes: float32(101), object(1)
memory usage: 1.0+ MB


None

In [42]:
from sklearn import preprocessing   
label_encoder = preprocessing.LabelEncoder() 
covid_df['Gender']= label_encoder.fit_transform(covid_df['Gender']) 
covid_df['Gender'] = covid_df['Gender'].astype(np.float64)

In [43]:
covid_df = covid_df.drop(covid_df.loc[:,'Lymp<1100':].columns, axis=1)

In [44]:
target = covid_df['Triage result']
train = covid_df.drop('Triage result', axis=1)

x_train, x_test, y_train, y_test = train_test_split(train, target, test_size=0.2, stratify=target, random_state=14)

print("Test: Number of each Class: " , sorted(Counter(y_test).items()), " Number of Samples: " ,len(x_test))
print("Train: Number of each Class: " , sorted(Counter(y_train).items()), " Number of Samples: " ,len(x_train))

Test: Number of each Class:  [(1.0, 225), (2.0, 261), (3.0, 30)]  Number of Samples:  516
Train: Number of each Class:  [(1.0, 899), (2.0, 1041), (3.0, 121)]  Number of Samples:  2061


In [45]:
columns_name = x_train.columns
scaler = MinMaxScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns=columns_name)
display(x_train.describe())

1,Age (yr),Gender,pulse rate,respiratory rate,Tempreture,Height,Weight,BMI,Systolic blood presure,Diastolic blood pressure,...,Prednisolone intake,Cyclophosphamide intake,other immunosuppression,pregnancy,delivery,HIV disease,Cardiovascular dis,non medical personnel,mediacl personnel,No risk factor
count,2061.000000,2061.000000,2061.000000,2061.000000,2061.000000,2061.000000,2061.000000,2061.000000,2061.000000,2061.000000,...,2061.000000,2061.0,2061.000000,2061.000000,2061.000000,2061.000000,2061.000000,2061.000000,2061.000000,2061.000000
mean,0.354823,0.553615,0.033949,0.321241,0.384414,0.504784,0.374082,0.316712,0.351685,0.399130,...,0.002911,0.0,0.014071,0.014556,0.001941,0.003396,0.018438,0.007278,0.065017,0.546822
std,0.169993,0.497238,0.030237,0.090860,0.127496,0.120547,0.135556,0.127852,0.119289,0.117006,...,0.053890,0.0,0.117812,0.119796,0.044022,0.058194,0.134560,0.085021,0.246616,0.497924
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.232558,0.000000,0.024444,0.269231,0.300000,0.413333,0.278261,0.231697,0.269737,0.321429,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.325581,1.000000,0.032593,0.307692,0.360000,0.506667,0.373913,0.305182,0.335526,0.392857,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.453488,1.000000,0.040741,0.384615,0.460000,0.586667,0.452174,0.389909,0.421053,0.473214,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [46]:
ros = RandomOverSampler(random_state=0)
x_train_ros, y_train_ros = ros.fit_resample(x_train, y_train)
print(sorted(Counter(y_train_ros).items()))

[(1.0, 1041), (2.0, 1041), (3.0, 1041)]


In [58]:
dataset = tf.data.Dataset.from_tensor_slices((x_train_ros.values, y_train_ros.values))

In [64]:
train_dataset = dataset.shuffle(len(x_train_ros)).batch(1)

In [72]:
def createCNN():
    tf.keras.backend.set_floatx('float64')
    model = Sequential()
    model.add(Dense(78, activation="relu"))
    model.add(Dense(50, activation="relu"))
    model.add(Dense(15, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation="softmax"))
    return model

In [73]:
learning_rate = 0.001
epochs = 15
batch_size = 64
optimizer = Adam(lr=learning_rate, decay=learning_rate / (epochs))

my_model = createCNN()


In [74]:
my_model.compile(optimizer=optimizer,
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
my_model_fit = my_model.fit(train_dataset, batch_size=batch_size, epochs=epochs, verbose=1)
my_model.summary()

Epoch 1/15
3123/3123 [==============================] - 2s 725us/step - loss: -0.6867 - accuracy: 0.3333
Epoch 2/15
3123/3123 [==============================] - 3s 827us/step - loss: -0.6867 - accuracy: 0.3333
Epoch 3/15
3123/3123 [==============================] - 3s 940us/step - loss: -0.6867 - accuracy: 0.3333
Epoch 4/15
3123/3123 [==============================] - 3s 1ms/step - loss: -0.6867 - accuracy: 0.3333
Epoch 5/15
3123/3123 [==============================] - 3s 996us/step - loss: -0.6867 - accuracy: 0.3333
Epoch 6/15
3123/3123 [==============================] - 3s 941us/step - loss: -0.6867 - accuracy: 0.3333 0s - loss: -0.6867 - accuracy: 0
Epoch 7/15
3123/3123 [==============================] - 3s 812us/step - loss: -0.6867 - accuracy: 0.3333
Epoch 8/15
3123/3123 [==============================] - 2s 706us/step - loss: -0.6867 - accuracy: 0.3333
Epoch 9/15
3123/3123 [==============================] - 2s 770us/step - loss: -0.6867 - accuracy: 0.3333
Epoch 10/15
3123/3123 [=